In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.io
from operator import itemgetter
import pandas as pd
from sklearn.cross_validation import train_test_split

import csv
import nupic

In [2]:
from nupic.swarming import permutations_runner
from nupic.swarming.permutations_runner import DEFAULT_OPTIONS
import nupic.swarming
from nupic.frameworks.opf.modelfactory import ModelFactory

In [16]:
# traindf = pd.read_csv('./lab_data/24chan/24chan_train1.csv', header = 0, skiprows = [1,2])
# testdf = pd.read_csv('./lab_data/24chan/24chan_test1.csv', header = 0, skiprows = [1,2])
traindf = pd.read_csv('./lab_data/24chan/24chan_train_full.csv', header = 0, skiprows = [1,2])
testdf = pd.read_csv('./lab_data/24chan/24chan_test_full.csv', header = 0, skiprows = [1,2])

print traindf.shape, testdf.shape
print traindf.head()

(100717, 24) (100603, 24)
        field1       field2       field3      field4      field5      field6  \
0 -3025.031006 -3898.197510 -5941.647461 -143.265717  223.767761 -896.394226   
1 -3025.013184 -3900.128662 -5938.768555 -139.367584  225.287674 -896.894958   
2 -3025.192139 -3902.274414 -5937.176758 -138.205292  225.430725 -896.054504   
3 -3022.474121 -3898.537109 -5932.116699 -136.184692  228.291748 -893.175598   
4 -3019.648682 -3890.043457 -5928.969238 -131.589172  232.779984 -891.941772   

         field7       field8        field9      field10       ...        \
0 -11081.028320 -9336.876953 -50944.109375 -8007.466797       ...         
1 -11080.598633 -9336.948242 -50917.480469 -8008.665039       ...         
2 -11080.205078 -9336.787109 -50951.457031 -8010.721191       ...         
3 -11071.533203 -9329.348633 -50996.410156 -8014.029297       ...         
4 -11068.529297 -9326.237305 -50985.449219 -8013.957520       ...         

        field16      field17      field18 

In [4]:
traindf.describe()

,field1,field2,field3,field4,field5,field6,field7,field8,field9,field10,...,field16,field17,field18,field19,field20,field21,field22,field23,field24,classification
count,100717.000000,100717.000000,100717.000000,100717.000000,100717.000000,100717.000000,100717.000000,100717.000000,100717.000000,100717.000000,...,100717.000000,100717.000000,100717.000000,100717.000000,100717.000000,100717.000000,100717.000000,100717.000000,100717.000000,100717.000000
mean,-3283.819059,-3658.182920,-5900.389288,192.092280,269.692334,-940.074065,-10884.284310,-8987.883506,-49181.831690,-8079.167572,...,-31609.668233,-1561.627804,1093.810430,-118.735434,-7244.681650,-3434.810036,-6818.156027,-13077.454052,-0.017879,1.003773
std,182.229787,129.751470,82.423759,165.729565,22.343574,33.076049,145.792570,177.230331,2157.864285,116.578124,...,763.169354,276.696774,31.736923,90.165789,90.657981,265.322635,90.795762,169.285106,0.000218,0.818492
min,-3648.394287,-3930.258789,-5998.581543,-147.789719,0.000000,-1015.770447,-11088.269531,-9336.948242,-52709.589844,-8252.370117,...,-32720.929688,-2177.864258,0.000000,-292.736298,-7302.904297,-3948.301025,-6918.311035,-13195.216797,-0.017881,0.000000
25%,-3427.881104,-3725.552490,-5934.494629,80.984833,255.596634,-946.354858,-10907.328125,-9095.442383,-50647.257812,-8131.259766,...,-32246.107422,-1761.764282,1063.907104,-181.013351,-7257.807129,-3633.856689,-6847.983398,-13136.958984,-0.017881,0.000000
50%,-3259.527588,-3633.695801,-5908.602539,186.091660,275.427094,-930.386780,-10868.275391,-8958.524414,-50310.500000,-8080.870117,...,-31580.687500,-1522.421875,1103.854126,-108.951332,-7244.289062,-3417.384521,-6814.706543,-13062.214844,-0.017881,1.000000
75%,-3127.008789,-3554.892578,-5866.777832,352.013123,286.960602,-919.443359,-10847.890625,-8881.312500,-47920.578125,-8019.554688,...,-31055.832031,-1318.949463,1114.672363,-35.494568,-7233.363281,-3209.996338,-6785.666992,-13027.596680,-0.017881,2.000000
max,0.000000,0.000000,0.000000,453.811890,314.462189,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1145.017090,28.306246,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000


In [5]:
#Number of fields without class label
num_fields = len(traindf.columns) - 1
print num_fields
# Exclude the column 'field14'
fields_names = ['field' + (str(i+1) if (i+1 < 14) else str(i+2)) for i in range(num_fields )] #+\
#    ['classification']
included_fields = []
for f in fields_names:
    field_dict = {'fieldName': f, 'fieldType': 'float', 'maxValue': 100000.0, 'minValue': -100000.0}
    included_fields.append(field_dict)

included_fields.append({'fieldName': 'classification', 'fieldType': 'int', 'maxValue': 2, 'minValue': 0})

23


In [6]:
SWARM_CONFIG = {
    'includedFields': included_fields,
    "streamDef": {
        "info": "classification_24channels",
        "version": 1,
        "streams": [
            {
                "info": "sine.csv",
                "source": "file:///home/polina/Documents/hse/course_work/lab_data/24chan/24chan_train_full.csv",
                "columns": [
                  "*"
                ]
            }
        ]
    },
    "inferenceType": "NontemporalClassification",
    "inferenceArgs": {
        "predictionSteps": [
          0
        ],
        "predictedField": u'classification'
    },
    "swarmSize": "medium"
}

In [7]:
def swarm_over_data(swarm_configuration):
    return nupic.swarming.permutations_runner.runWithConfig(swarm_configuration, {'maxWorkers': 8, 'overwrite': True})


#input_file = "./lab_data/24chan/24chan_train1.csv"
model_params = swarm_over_data(SWARM_CONFIG)

Generating experiment files in directory: /home/polina/Documents/hse/course_work...
Writing  313 lines...
Writing  114 lines...
done.
None
Successfully submitted new HyperSearch job, jobID=1020
##>> UPDATED WORKER STATE: 
{   u'activeSwarms': [   u'modelParams|sensorParams|encoders|field8',
                         u'modelParams|sensorParams|encoders|field9',
                         u'modelParams|sensorParams|encoders|field6',
                         u'modelParams|sensorParams|encoders|field7',
                         u'modelParams|sensorParams|encoders|field4',
                         u'modelParams|sensorParams|encoders|field5',
                         u'modelParams|sensorParams|encoders|field2',
                         u'modelParams|sensorParams|encoders|field3',
                         u'modelParams|sensorParams|encoders|field1',
                         u'modelParams|sensorParams|encoders|field18',
                         u'modelParams|sensorParams|encoders|field19',
      

In [8]:
model = ModelFactory.create(model_params)
model.enableInference({"predictedField": 'classification'})

In [66]:
model.save("/home/polina/Documents/hse/course_work/lab_data/24chan/model24_1")
#model.load("/home/polina/Documents/hse/course_work/lab_data/24chan/model24_1")

In [9]:
# Function return dataframe with 2 columns: classification value and predicted values,
# obtained with model.
# Parameters:
#  -- data_frame - pandas df
#  -- class_column - column from data_frame with class labels
#  -- model - model, obtained with nupic (class <nupic.frameworks.opf.clamodel.CLAModel>)
def predicted_values(data_frame, class_column, model):
    output = zip(np.asarray(class_column), np.zeros(len(class_column), dtype = int))
    output = pd.DataFrame(output)
    input_dict = data_frame.to_dict(orient = "index")
    for k in input_dict.keys():
        row = input_dict[k]
        result = model.run(row)
        output[1][k] = result.inferences['multiStepBestPredictions'][0]
    return output

def mean_error(init_class, predicted_class):
    return np.mean(np.array(init_class) != np.array(predicted_class))

In [17]:
test_output = predicted_values(testdf, testdf['classification'], model)
train_output =  predicted_values(traindf, traindf['classification'], model)
print test_output.head()
print train_output.head()

   0  1
0  1  0
1  1  0
2  1  0
3  1  0
4  1  0
   0  1
0  1  0
1  1  0
2  1  0
3  1  0
4  1  0


In [19]:
print np.unique(test_output[1])
print np.unique(train_output[1])

[  0 127 254]
[  0 127 254]


In [20]:
test_output[1] = test_output[1] // 127
train_output[1] = train_output[1] // 127

In [24]:
#print output.head()
print np.sum(np.array(test_output[1]) != np.array(testdf['classification']))
print test_output.shape
print np.array(test_output[1]) != np.array(testdf['classification'])

39
(100603, 2)
[ True  True  True ..., False False False]


In [25]:
train_error = mean_error(train_output[0], train_output[1])
test_error  = mean_error(test_output[0], test_output[1])
print train_error, test_error

0.000883664128201 0.000387662395754


### Try to change min/max borders

In [38]:
print "MIN and MAX values for each column"
for i in traindf.columns:
    print '%s: train %.1f , %.1f ; test %.1f , %.1f' % \
    (i, min(traindf[i]), max(traindf[i]), min(testdf[i]), max(testdf[i]))

MIN and MAX values for each column
field1: train -3648.4 , 0.0 ; test -3648.0 , 0.0
field2: train -3930.3 , 0.0 ; test -3928.1 , 0.0
field3: train -5998.6 , 0.0 ; test -5998.2 , 0.0
field4: train -147.8 , 453.8 ; test -146.4 , 452.9
field5: train 0.0 , 314.5 ; test 0.0 , 314.5
field6: train -1015.8 , 0.0 ; test -1015.8 , 0.0
field7: train -11088.3 , 0.0 ; test -11088.3 , 0.0
field8: train -9336.9 , 0.0 ; test -9330.6 , 0.0
field9: train -52709.6 , 0.0 ; test -52484.6 , 0.0
field10: train -8252.4 , 0.0 ; test -8252.4 , 0.0
field11: train 0.0 , 2848.2 ; test 0.0 , 2848.2
field12: train -6205.8 , 0.0 ; test -6205.8 , 0.0
field13: train 0.0 , 2256.1 ; test 0.0 , 2252.6
field15: train -13038.6 , 0.0 ; test -13036.3 , 0.0
field16: train -32720.9 , 0.0 ; test -32720.9 , 0.0
field17: train -2177.9 , 0.0 ; test -2144.5 , 0.0
field18: train 0.0 , 1145.0 ; test 0.0 , 1145.0
field19: train -292.7 , 28.3 ; test -291.8 , 28.3
field20: train -7302.9 , 0.0 ; test -7302.3 , 0.0
field21: train -3948.3 ,

In [44]:
#Number of fields without class label
num_fields = len(traindf.columns) - 1
print num_fields
# Exclude the column 'field14'
fields_names = ['field' + (str(i+1) if (i+1 < 14) else str(i+2)) for i in range(num_fields )] #+\
# ['classification']
included_fields = []
for f in fields_names:
    field_dict = {'fieldName': f, 'fieldType': 'float', \
                  'maxValue': 100000.0 if max(traindf[f]) > 0. else 0., \
                  'minValue': -100000.0 if min(traindf[f]) < 0. else 0.}
    included_fields.append(field_dict)

included_fields.append({'fieldName': 'classification', 'fieldType': 'int', 'maxValue': 2, 'minValue': 0})

23


In [45]:
SWARM_CONFIG_1 = SWARM_CONFIG

SWARM_CONFIG1['includedFields'] = included_fields

In [58]:
# model_params_1 = swarm_over_data(SWARM_CONFIG_1)
# model1 = ModelFactory.create(model_params_1)
# model1.enableInference({"predictedField": 'classification'})
model1.save("/home/polina/Documents/hse/course_work/lab_data/24chan/model24_strictminmax")
#model1.load("/home/polina/Documents/hse/course_work/lab_data/24chan/model24_strictminmax")

test_output = predicted_values(testdf, testdf['classification'], model1)
train_output =  predicted_values(traindf, traindf['classification'], model1)
print test_output.head()
print train_output.head()

   0  1
0  1  0
1  0  0
2  1  0
3  2  0
4  1  0
   0    1
0  1    0
1  1    0
2  1    0
3  1  127
4  1  127


In [59]:
print np.unique(test_output[1])
print np.unique(train_output[1])

[  0 127 254]
[  0 127 254]


In [60]:
test_output[1] = test_output[1] // 127
train_output[1] = train_output[1] // 127

In [61]:
#print output.head()
print np.sum(np.array(test_output[1]) != np.array(testdf['classification']))
print test_output.shape
print np.array(test_output[1]) != np.array(testdf['classification'])

train_error = mean_error(train_output[0], train_output[1])
test_error  = mean_error(test_output[0], test_output[1])
print train_error, test_error

20144
(33237, 2)
[ True False  True ...,  True  True  True]
0.000794304834338 0.60607154677
